In [45]:
from rtree import index
import face_recognition
import pickle
import pandas as pd
import numpy as np
from queue import PriorityQueue

scaler_path = "./bin/scaler.dat"
pca_path = "./bin/pca.dat"
ncomponents_path = "./bin/ncomponents.dat"

scaler = pickle.load(open(scaler_path, "rb"))
pca = pickle.load(open(pca_path, "rb"))
ncomponents = pickle.load(open(ncomponents_path, "rb"))
df = pd.read_csv("./data/datasetv2.csv")

In [24]:
features = [str(i) for i in range(1, ncomponents+1)]
collection = df.loc[:, features]
y = df.loc[:, ["path"]]
y.tail(5)

,path
14517,Roger_Moore/Roger_Moore_0002.jpg
14518,Roger_Moore/Roger_Moore_0004.jpg
14519,Infanta_Cristina/Infanta_Cristina_0001.jpg
14520,Iva_Majoli/Iva_Majoli_0002.jpg
14521,Iva_Majoli/Iva_Majoli_0001.jpg


In [44]:
def generate_df(x):
    return pd.DataFrame(data=x, columns = [str(i) for i in range(1, 129)])

def ED(v1, v2):
    return np.linalg.norm(v1-v2)

def range_seach(collection, query, radio):
    result = []
    nrows = collection.shape[0]
    for i in range(nrows):
        dist = ED(collection.iloc[i].values, query)
        if(dist < radio):
            image_path = y.iloc[i].values[0]
            result.append((dist, image_path))
    return result

def parser_image(image_path):
    picture = face_recognition.load_image_file(image_path)    
    all_face_encodings = face_recognition.face_encodings(picture)
    x = generate_df(all_face_encodings)
    x_scaled = scaler.transform(x)
    x_pca = pca.transform(x_scaled)
    return x_pca[0]

def searchKNN_sequential(collection, query, k):
    pq = PriorityQueue()
    nrows = collection.shape[0]
    for i in range(nrows):
        dist = ED(collection.iloc[i].values, query) * (-1)
        image_path = y.iloc[i].values[0]
        if(pq.qsize() < k):
            pq.put((dist, image_path))
        else:
            top = pq.get()
            if(dist > top[0]):
                pq.put((dist, image_path))
            else:
                pq.put(top)
    result = [0] * k
    i = k - 1
    while not pq.empty():
        data = pq.get()
        result[i] = data
        i -= 1
    return result

In [39]:
image_path = "test/Roger_Moore_0004.jpg"
x_pca = parser_image(image_path)


In [49]:
radio = 10 # [9, 11]
result = range_seach(collection, x_pca, radio)
#result = searchKNN_sequential(collection, x_pca, 6)
print(len(result))

5


In [48]:
for item in result:
    print(item[1])

Roger_Moore/Roger_Moore_0004.jpg
Roger_Moore/Roger_Moore_0001.jpg
Roger_Moore/Roger_Moore_0003.jpg
Roger_Moore/Roger_Moore_0002.jpg
Enrique_Bolanos/Enrique_Bolanos_0003.jpg
Roger_Moore/Roger_Moore_0005.jpg
